Advanced Convolutional Neural Networks (CNNs) with CIFAR-10 dataset
=========

In this tutorial we will learn how to use more complex CNNs, showing that the training of a __deeper__ CNN can improve the performance of the model. We will also explore the concept of __data augmentation__ to understand how to increase the variability of the training set by, for example, rotating the original images to generate new training stimuli.

This tutorail will use the CIFAR-10 training set.


**CNN for CIFAR-10**

To work with more complex CNNs, we will now use a more complex training dataset called __CIFAR-10__. https://www.cs.toronto.edu/~kriz/cifar.html . CIFAR-10 is a benchamark machine learning set of low-resolution, colour images. It includes 60000 32x32 colour (using 3 RGB colour channels) images in these 10 classes of objects: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck. Each class has 6000. There are 50000 training images and 10000 test images. This dataset is enclosed in the default Anaconda KERAS package. 


**Initialisation of the program**

The program starts with the importing of typical Keras and other Python service modules. 

In [1]:
# importing of modules for CIFAR-10 CNN 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import LearningRateScheduler

# importing of service libraries
import numpy as np
import matplotlib.pyplot as plt

print('Libraries imported.')

2024-04-25 02:48:39.162581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 02:48:39.162607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 02:48:39.163637: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 02:48:39.169177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 02:48:39.809936: W tensorflow/compiler/tf2

Libraries imported.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [3]:
import tensorflow as tf

# Check CUDA
print("Is CUDA available? :", tf.test.is_built_with_cuda())

# Check cuDNN
print("Is cuDNN available? :", tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

Is CUDA available? : True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is cuDNN available? : True


2024-04-25 02:48:40.793839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:40.860577: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:40.860762: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
import tensorflow as tf

# This line ensures TensorFlow will use the GPU if available
# tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2024-04-25 02:48:40.943731: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:40.943927: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:40.944054: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available, using CPU instead.")

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


The following constant and variable definitions are needed for the network and training parameters.


In [6]:
#training constants
BATCH_SIZE = 128
N_EPOCH = 40 # use 20 for best initial results
N_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
# OPTIM = RMSprop()

print('Main variables initialised.')

Main variables initialised.


Constant definition for the training set images

In [7]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

print('Image variables initialisation')

Image variables initialisation


__CIFAR-10 data loading and processing__

Loading and preparation of the CIFAR-10 training set.

In [8]:
#load dataset
(input_X_train, output_y_train), (input_X_test, output_y_test) = cifar10.load_data()
print('input_X_train shape:', input_X_train.shape)
print(input_X_train.shape[0], 'train samples')
print(input_X_test.shape[0], 'test samples')
 
# convert to categorical
output_Y_train = utils.to_categorical(output_y_train, N_CLASSES)
output_Y_test = utils.to_categorical(output_y_test, N_CLASSES) 

# float and normalization
input_X_train = input_X_train.astype('float32')
input_X_test = input_X_test.astype('float32')
input_X_train /= 255
input_X_test /= 255


input_X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Data Augmentation
-------------

To further improve the performance of the model, it is advisable to use a larger training set, to expose the network to more variations of the images.
One way to achieve this, without having to collect new images from the real world, is to __augment__ the existing images with multiple types of transformations of the dataset stimuli. This can include rotation of the image, rescaling, horizontal/vertical flip, zooming, channel shift, etc.

Below is an example of the code that augments the current datase.


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.datasets import cifar10

#load dataset
#(input_X_train, output_y_train), (input_X_test, output_y_test) = cifar10.load_data()

# augumenting
print("Augmenting training set images...")
    
datagen = ImageDataGenerator(
   rotation_range=20,
   width_shift_range=0.2,
   height_shift_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   fill_mode='nearest')

# rotation_range is a value in degrees (0 - 180) for randomly rotating pictures
# width_shift and height_shift are ranges for randomly translating pictures vertically or horizontally
# zoom_range is for randomly zooming pictures 
# horizontal_flip is for randomly flipping the images horizontally
# fill_mode fills in new pixels that can appear after a rotation or a shift


Augmenting training set images...


In [10]:
from sklearn.model_selection import train_test_split

steps_per_epoch = input_X_train.shape[0] // BATCH_SIZE

# Split the data into 80% training and 20% validation
input_X_train_split, input_X_val, output_Y_train_split, output_Y_val = train_test_split(
    input_X_train, output_Y_train, test_size=0.2, random_state=42
)

In [11]:
# # Create a generator object
# train_generator = datagen.flow(input_X_train, output_Y_train, batch_size=BATCH_SIZE)

# Creating the training generator
train_generator = datagen.flow(
    input_X_train_split, 
    output_Y_train_split, 
    batch_size=BATCH_SIZE
)

# Creating the validation generator
validation_generator = datagen.flow(
    input_X_val, 
    output_Y_val, 
    batch_size=BATCH_SIZE
)

# Test generator output
x_batch, y_batch = next(train_generator)
print("Batch x shape:", x_batch.shape)
print("Batch y shape:", y_batch.shape)

Batch x shape: (128, 32, 32, 3)
Batch y shape: (128, 10)


A deeper CNN
-------------

To improve the performance of the network on the CIFAR-10 dataset, it is possible to use a deeper CNN, with a chain of multiple convolution and pooling layers.
The following network will be used:

conv+conv+maxpool+dropout+conv+conv+maxpool

The final classification layers will use the standard:

dense+dropout+dense

All the layers will use the reLu function, except the final one with the Softmax function necessary for the categorical classification 


In [12]:
# # REUSE THE SAME INITIALISATION CODE AND THE TRAINING AND TEST DATA SET LOADING OPERATION

from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, LeakyReLU

# Complex DNN model definition
# model = Sequential()
 
# model.add(Conv2D(128, kernel_size=3, padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
# model.add(Activation('relu'))
# model.add(Conv2D(128, kernel_size=3, padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
 
# model.add(Conv2D(128, kernel_size=3, padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, 3, 3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
 
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(N_CLASSES))
# model.add(Activation('softmax'))


# # OPTIM = SGD(lr=0.01)
# # OPTIM = RMSprop(lr=0.01)
# OPTIM = Adam(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

# model.summary()

In [13]:
# # Define model architecture
# model = Sequential([
#     Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(64, (3, 3), padding='same'),
#     Activation('relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),  # Slightly reduced dropout

#     Conv2D(128, (3, 3), padding='same'),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(128, (3, 3)),
#     Activation('relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.3),  # Slightly reduced dropout

#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.4),  # Slightly reduced dropout
#     Dense(N_CLASSES, activation='softmax')
# ])

# # Compile the model
# optimizer = Adam(lr=0.001)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# # Callbacks
# callbacks = [
#     EarlyStopping(monitor='val_accuracy', patience=5, verbose=1),
#     ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
# ]

# # Simplified data augmentation
# data_generator = ImageDataGenerator(
#     rotation_range=10,  # Less aggressive rotation
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True
# )

# # Model summary
# model.summary()

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, LeakyReLU

model = Sequential()

# First convolutional block
model.add(Conv2D(32, kernel_size=3, padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=3, padding='same'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolutional block
model.add(Conv2D(64, kernel_size=3, padding='same'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding='same'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolutional block
model.add(Conv2D(128, kernel_size=3, padding='same'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=3, padding='same'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten and dense layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(N_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                        

2024-04-25 02:48:42.022700: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:42.022890: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-25 02:48:42.023018: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

**Training of the deeper CNN**

Let's train (fit) this new model.



In [ ]:
# training/fitting of the complex DNN model 
# history = model.fit(input_X_train, output_Y_train, batch_size=BATCH_SIZE, epochs=N_EPOCH, validation_split=VALIDATION_SPLIT,  verbose=VERBOSE)

history = model.fit(
    train_generator,
    steps_per_epoch=len(input_X_train_split) // BATCH_SIZE,  # Correct calculation for steps per epoch
    validation_data=validation_generator,
    validation_steps=len(input_X_val) // BATCH_SIZE,  # Correct calculation for validation steps
    epochs=N_EPOCH,
    verbose=VERBOSE
)

Epoch 1/40


2024-04-25 02:48:43.214008: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-25 02:48:43.394406: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-25 02:48:43.493939: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-25 02:48:43.884085: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-25 02:48:44.560719: I external/local_xla/xla/service/service.cc:168] XLA service 0x751e08bf4640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-25 02:48:44.560741: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 20

312/312 [==============================] - 22s 58ms/step - loss: 1.9204 - accuracy: 0.2785 - val_loss: 1.6746 - val_accuracy: 0.3766
Epoch 2/40
312/312 [==============================] - 17s 55ms/step - loss: 1.6077 - accuracy: 0.4050 - val_loss: 1.4352 - val_accuracy: 0.4606
Epoch 3/40
312/312 [==============================] - 18s 56ms/step - loss: 1.4427 - accuracy: 0.4708 - val_loss: 1.3497 - val_accuracy: 0.5049
Epoch 4/40
312/312 [==============================] - 17s 56ms/step - loss: 1.3382 - accuracy: 0.5157 - val_loss: 1.2239 - val_accuracy: 0.5561
Epoch 5/40
312/312 [==============================] - 17s 56ms/step - loss: 1.2760 - accuracy: 0.5403 - val_loss: 1.1503 - val_accuracy: 0.5812
Epoch 6/40
312/312 [==============================] - 17s 56ms/step - loss: 1.2069 - accuracy: 0.5685 - val_loss: 1.0889 - val_accuracy: 0.6139
Epoch 7/40
312/312 [==============================] - 17s 55ms/step - loss: 1.1564 - accuracy: 0.5889 - val_loss: 1.0635 - val_accuracy: 0.6199
Epo

**Analysis of the Deeper CNN results**

This generates the test scores and plots for the new, deeper DNN.


In [ ]:
#Testing
score = model.evaluate(input_X_test, output_Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score/loss:", score[0])
print('Test accuracy:', score[1])

# Set up a larger figure to accommodate both subplots side by side
plt.figure(figsize=(10, 4))  # Adjust the size as needed

# First subplot for accuracy
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

# Second subplot for loss
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# Display the figure with subplots
plt.tight_layout()
plt.show()

Data Augmentation
-------------

To further improve the performance of the model, it is advisable to use a larger training set, to expose the network to more variations of the images.
One way to achieve this, without having to collect new images from the real world, is to __augment__ the existing images with multiple types of transformations of the dataset stimuli. This can include rotation of the image, rescaling, horizontal/vertical flip, zooming, channel shift, etc.

Below is an example of the code that augments the current datase.




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.datasets import cifar10

#load dataset
#(input_X_train, output_y_train), (input_X_test, output_y_test) = cifar10.load_data()

# augumenting
print("Augmenting training set images...")
    
datagen = ImageDataGenerator(
   rotation_range=20,
   width_shift_range=0.2,
   height_shift_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   fill_mode='nearest')

# rotation_range is a value in degrees (0 - 180) for randomly rotating pictures
# width_shift and height_shift are ranges for randomly translating pictures vertically or horizontally
# zoom_range is for randomly zooming pictures 
# horizontal_flip is for randomly flipping the images horizontally
# fill_mode fills in new pixels that can appear after a rotation or a shift


In [ ]:
print("Number of training samples:", input_X_train.shape[0])
print("Batch size:", BATCH_SIZE)

In [ ]:
steps_per_epoch = input_X_train.shape[0] // BATCH_SIZE
print("Steps per epoch:", steps_per_epoch)

In [ ]:
# # Create a generator object
# train_generator = datagen.flow(input_X_train, output_Y_train, batch_size=BATCH_SIZE)

# # Test generator output
# x_batch, y_batch = next(train_generator)
# print("Batch x shape:", x_batch.shape)
# print("Batch y shape:", y_batch.shape)

In [ ]:
# history = model.fit(
#     train_generator,
#     steps_per_epoch=steps_per_epoch,  # Verify this is calculated correctly
#     epochs=N_EPOCH,
#     verbose=VERBOSE
# )

__Training with augmented data__

The function below used the dynamic generation of the augmented data during the training (just in time).

In [ ]:

# #fit the dataset
# datagen.fit(input_X_train)


# # train by fitting the model on batches with real-time data augmentation
# history = model.fit(datagen.flow(input_X_train, output_Y_train, batch_size=BATCH_SIZE), steps_per_epoch=input_X_train.shape[0]//BATCH_SIZE, epochs=N_EPOCH, verbose=VERBOSE)



**Analysis of the Data Augmented, Deeper CNN results**

This generates the test scores and plots for the deeper DNN trained on the augmented data.

In [ ]:
#Testing
score = model.evaluate(input_X_test, output_Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score/loss:", score[0])
print('Test accuracy:', score[1])

# list all data in history
print(history.history.keys())

# summarize history for accuracy
#plt.plot(mo)
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



Below is a commented different example of a data augmentation approach. 

But we have carried out plenty of slow, long simulations for this class, and we can stop here.

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(input_X_train)

Conclusions
-------------

Today we learned to train more complex DNNs, and to use data augmentation to further improve the network training and performance.

**Copyright (c)** 2022 Angelo Cangelosi, MIT License. Code and examples adapted from Gulli & Pal (2017) Deep Learning with Kera. Punkt Publishing. With support from Wenjie Huang.